##How to get rid of Loops in Python using NumPy

The below are the snippets of code during the [talk](https://www.youtube.com/watch?v=EEUXKG97YRw) by Jake VanderPlas

In [1]:
print "Hello World!"

Hello World!


In [2]:
# A Silly function implemented in Python
def func_python(N):
    d = 0.0
    for i in range(N):
        d += ( i % 3 - 1) * i
    return d

In [3]:
# Use IPython timit magic to time the execution
%timeit func_python(10000)

1000 loops, best of 3: 1.63 ms per loop


%%fortran
subroutine func_fort(n,d):
        integer, intent(in) :: n
        double precision, intent(out) :: d
        integer :: i
        d = 0
        do i = 0, n-1
            d = d + (mod(i,3) - 1)*i
        end do
end subroutine func_fort

###Strategy \#1: 
Use NumPy's Ufuncs : For element-wise operations

In [4]:
#Element-Wise Operations with Python lists:
a = [1, 3, 2, 4, 3, 1, 4, 2]
b = [val + 1 for val in a]

print b

[2, 4, 3, 5, 4, 2, 5, 3]


In [5]:
import numpy as np

In [6]:
#with NumPy arrays:
a = np.array(a)

b = a + 5 #element-wise
print (b)

[6 8 7 9 8 6 9 7]


In [7]:
a = list(range(100000))
%timeit [val + 5 for val in a]

100 loops, best of 3: 6.41 ms per loop


In [8]:
#Ufuncs are fast
a = np.array(a)
%timeit a + 5

The slowest run took 8.68 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 78 µs per loop


###Strategy \#2:
Use NumPy's aggregations: For Array Summarizations.

In [9]:
from random import random

c = [random() for i in range(100000)]
%timeit min(c)

100 loops, best of 3: 2.4 ms per loop


In [10]:
c = np.array(c)
%timeit c.min()

10000 loops, best of 3: 44.7 µs per loop


**NumPy aggregations also work on multi-dimensional arrays**

In [11]:
M = np.random.randint(0,10,(3,5))
M

array([[5, 7, 5, 1, 4],
       [3, 7, 2, 4, 8],
       [4, 9, 7, 5, 1]])

In [12]:
M.sum()

72

In [13]:
M.sum(axis=0)

array([12, 23, 14, 10, 13])

In [14]:
M.sum(axis=1)

array([22, 24, 26])

####Lots of aggregations available

np.min(), np.max(), np.sum(), np.prod(), np.mean(), np.std(), np.var(), np.any(), np.all(), np.median(), np.percentile() -- These are straight forward agg functions
np.argmin() & np.argmax() --gives the min and max index no. of the array
np.nanmin(), np.nanmax() & np.nansum() -gives the min, max and sum ignoring the nan

###Strategy \#3:
Use NumPy's Broadcasting: For Combining Arrays of different sizes

Broadcasting is a set of rules by which ufuncs operate on arrays of different sizes and/or dimensions

In [15]:
np.arange(3)

array([0, 1, 2])

In [16]:
np.arange(3)-5

array([-5, -4, -3])

In [17]:
np.ones((3,3))

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 1.,  1.,  1.]])

In [18]:
np.ones((3,3)) - np.arange(3)

array([[ 1.,  0., -1.],
       [ 1.,  0., -1.],
       [ 1.,  0., -1.]])

In [19]:
np.arange(3).reshape((3,1))

array([[0],
       [1],
       [2]])

In [20]:
np.arange(3).reshape((3,1)) - np.arange(3)

array([[ 0, -1, -2],
       [ 1,  0, -1],
       [ 2,  1,  0]])

###Strategy \#4:
Use NumPy's slicing, masking and fancy indexing: To Access parts of the Arrays very Quickly

With Python Lists, indexing accepts integers or slices..

In [21]:
L = [2, 3, 5, 7, 11]
L[0] #integer index

2

In [22]:
L[1:3] #slice for multiple elements

[3, 5]

NumPy arrays are similar

In [23]:
L = np.array(L)
L

array([ 2,  3,  5,  7, 11])

In [24]:
L[0]

2

In [25]:
L[1:3]

array([3, 5])

..but NumPy offers other fast and convenient indexing options as well

####Masking: indexing with boolean marks

In [26]:
L

array([ 2,  3,  5,  7, 11])

A Mask is a boolean array:

In [27]:
mask = np.array([False, True, True,
                False, True])

L[mask]

array([ 3,  5, 11])

In [28]:
mask = (L < 4) | (L > 8) #"|" = "bitwise OR"
L[mask]

array([ 2,  3, 11])

####Fancy Indexing: passing a list/array of indices

In [29]:
ind=[0, 4, 2]
L[ind]

array([ 2, 11,  5])

In [30]:
####Multiple dimensions: use commas to separate indices!
M = np.arange(6).reshape(2,3)
M

array([[0, 1, 2],
       [3, 4, 5]])

In [31]:
#multiple indices seperated by comma
M[0,1]

1

In [32]:
#mixing slices 
M[:, 1]

array([1, 4])

In [33]:
#masking  the full array
M[abs(M-3) < 2]

array([2, 3, 4])

In [34]:
#mixing fancy indexing and slicing
M[[1, 0], :2]

array([[3, 4],
       [0, 1]])

In [35]:
#mixing masking and slicing
M[M.sum(axis=1)>4, 1:]

array([[4, 5]])

###Example: To Compute K- Nearest Neighbours in NumPy

In [36]:
#1000 points in 3 dimensions
X = np.random.random((1000,3))
X 

array([[ 0.34878945,  0.26047918,  0.89074719],
       [ 0.36829889,  0.46677968,  0.07431446],
       [ 0.23901422,  0.27642177,  0.05514388],
       ..., 
       [ 0.21883228,  0.24787516,  0.82290999],
       [ 0.31043767,  0.82171798,  0.8682368 ],
       [ 0.24670232,  0.26792003,  0.57591205]])

In [37]:
X.shape

(1000L, 3L)

In [38]:
# Broadcasting to find pair-wise differences
X.reshape(1000,1,3).shape

(1000L, 1L, 3L)

In [39]:
diff = X.reshape(1000,1,3) - X
diff

array([[[ 0.        ,  0.        ,  0.        ],
        [-0.01950944, -0.2063005 ,  0.81643274],
        [ 0.10977523, -0.01594259,  0.83560331],
        ..., 
        [ 0.12995718,  0.01260402,  0.06783721],
        [ 0.03835178, -0.5612388 ,  0.02251039],
        [ 0.10208714, -0.00744086,  0.31483514]],

       [[ 0.01950944,  0.2063005 , -0.81643274],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.12928467,  0.19035792,  0.01917057],
        ..., 
        [ 0.14946662,  0.21890452, -0.74859553],
        [ 0.05786122, -0.35493829, -0.79392235],
        [ 0.12159657,  0.19885965, -0.50159759]],

       [[-0.10977523,  0.01594259, -0.83560331],
        [-0.12928467, -0.19035792, -0.01917057],
        [ 0.        ,  0.        ,  0.        ],
        ..., 
        [ 0.02018194,  0.02854661, -0.7677661 ],
        [-0.07142345, -0.54529621, -0.81309292],
        [-0.0076881 ,  0.00850173, -0.52076817]],

       ..., 
       [[-0.12995718, -0.01260402, -0.06783721],
        

In [40]:
diff.shape

(1000L, 1000L, 3L)

In [41]:
#Aggregate to find pairwise distances
D = (diff ** 2).sum(2)
D.shape

(1000L, 1000L)

In [42]:
# To avoid the same points as the distance between them is zero.
# set diagonal to infinity to skip self-neigbours
i = np.arange(1000)
D[i,i] = np.inf

In [43]:
# Print the indices of the nearest Neighbours 
i = np.argmin(D, 1)
print i[:10]

[699 713 223 454 253 964 145  61 718 711]


In [44]:
# double check with scikit-learn
from sklearn.neighbors import NearestNeighbors
d, i = NearestNeighbors().fit(X).kneighbors(X,2)
print i[:10]

[[  0 699]
 [  1 713]
 [  2 223]
 [  3 454]
 [  4 253]
 [  5 964]
 [  6 145]
 [  7  61]
 [  8 718]
 [  9 711]]
